In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

import twitter

consumer_key = config['TWITTER']['CK']
consumer_secret = config['TWITTER']['CS']
access_token_key = config['TWITTER']['ATK']
access_token_secret = config['TWITTER']['ATS']

# set to extended mode and sleep if exceed rate limit
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token_key,
                  access_token_secret=access_token_secret,
                  tweet_mode= 'extended',
                  sleep_on_rate_limit=True)

In [ ]:
db_twitter = client["Twitter"]

# Get full_text of the truncated tweets in twitter-australia collection

In [ ]:
# get number to limit the records from mongodb
limit_number = int(config['TWITTER']['Number-Truncated-Tweets'])

# get the first "limit_number" records from "twitter-australia" collection
for document in db_twitter["twitter-test"].find({"truncated":True}).limit(limit_number):
        try:
            # get full_text using twitter api
            full_text = api.GetStatus(document["id_str"]).full_text
            #print("--------------\n" + full_text)

            # update fields(add full_text if exist and change truncated value to false)
            db_twitter["twitter-test"].update({"_id": document["_id"]}, {"$set": {"full_text": full_text, "truncated":False}})
        except:
            db_twitter["twitter-test"].update({"_id": document["_id"]}, {"$set": {"full_text": None, "truncated":False}})   
            #print("--------------\n" + "exception: pass")
            
print("Done: get full_text of " + str(limit_number) + " truncated tweets in twitter-australia collection.")


# Get full_text of the truncated tweets in twitter-other collection

In [ ]:
# get the first "limit_number" records from "twitter-other" collection
for document in db_twitter["twitter-other"].find({"truncated":True}).limit(limit_number):
        try:
            # get full_text using twitter api
            full_text = api.GetStatus(document["id_str"]).full_text
            #print("--------------\n" + full_text)

            # update fields(add full_text if exist and change truncated value to false)
            db_twitter["twitter-other"].update({"_id": document["_id"]}, {"$set": {"full_text": full_text, "truncated":False}})
        except:
            db_twitter["twitter-other"].update({"_id": document["_id"]}, {"$set": {"full_text": None, "truncated":False}})   
            #print("--------------\n" + "exception: pass")
            
print("Done: get full_text of " + str(limit_number) + " truncated tweets in twitter-other collection.")